In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from layer import KernelConv2d
%matplotlib inline
def mkdirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

## Kervolution LeNet

In [5]:
class KNNet(nn.Module):
    def __init__(self):
        super(KNNet,self).__init__()
        self.conv1=KernelConv2d(1,10,5)
        print(self.conv1)
        self.bn1=nn.BatchNorm2d(10)
        self.conv2=KernelConv2d(10,20,5)
        self.bn2=nn.BatchNorm2d(20)
        self.conv2_drop=nn.Dropout2d()
        self.fc1=nn.Linear(320,50)
        self.fc2=nn.Linear(50,10)
    def forward(self,x):
        x=F.relu(F.max_pool2d(self.conv1(x),2))
        x=self.bn1(x)
        x=F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x=self.bn2(x)
        x=x.view(-1,320)
        x=F.relu(self.fc1(x))
        x=F.dropout(x,training=self.training)
        x=F.relu(self.fc2(x))
        return F.log_softmax(x,dim=1)

In [6]:
train_loader=torch.utils.data.DataLoader(
    datasets.MNIST("data",train=True,download=True,transform=transforms.Compose([
                transforms.ToTensor(),
            ])),batch_size=128,shuffle=True)
test_loader=torch.utils.data.DataLoader(
    datasets.MNIST("data",train=False,download=True,transform=transforms.Compose([
                transforms.ToTensor(),
            ])),batch_size=128,shuffle=False
)
attack_test_loader=torch.utils.data.DataLoader(
    datasets.MNIST("data",train=False,download=True,transform=transforms.Compose([
                transforms.ToTensor(),
            ])),batch_size=1,shuffle=False
)
print(len(train_loader))
print(len(test_loader))

469
79


In [7]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

knn=KNNet().to(device)
knn.train(mode=True)

KernelConv2d(
  1, 10, kernel_size=(5, 5), stride=(1, 1), bias=False
  (kernel_fn): PolynomialKernel()
)


KNNet(
  (conv1): KernelConv2d(
    1, 10, kernel_size=(5, 5), stride=(1, 1), bias=False
    (kernel_fn): PolynomialKernel()
  )
  (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): KernelConv2d(
    10, 20, kernel_size=(5, 5), stride=(1, 1), bias=False
    (kernel_fn): PolynomialKernel()
  )
  (bn2): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [8]:
criterion=torch.nn.NLLLoss()

In [9]:
def compute_accuray(pred,true):
    pred_idx=pred.argmax(dim=1).detach().cpu().numpy()
    tmp=pred_idx==true.cpu().numpy()
    return sum(tmp)/len(pred_idx)
def train(m,out_dir):
    iter_loss=[]
    train_losses=[]
    test_losses=[]
    iter_loss_path=os.path.join(out_dir,"iter_loss.csv")
    epoch_loss_path=os.path.join(out_dir,"epoch_loss.csv")
    nb_epochs=20
    last_loss=99999
    mkdirs(os.path.join(out_dir,"models"))
    optimizer=optim.SGD(m.parameters(),lr=0.003,momentum=0.9)
    for epoch in range(nb_epochs):
        train_loss=0.
        train_acc=0.
        m.train(mode=True)
        for data,target in train_loader:
            data,target=data.to(device),target.to(device)
            optimizer.zero_grad()
            output=m(data)
            loss=criterion(output,target)
            loss_value=loss.item()
            iter_loss.append(loss_value)
            train_loss+=loss_value
            loss.backward()
            optimizer.step()
            acc=compute_accuray(output,target)
            train_acc+=acc
        train_losses.append(train_loss/len(train_loader))
        
        test_loss=0.
        test_acc=0.
        m.train(mode=False)
        for data,target in test_loader:
            data,target=data.to(device),target.to(device)
            output=m(data)
            loss=criterion(output,target)
            loss_value=loss.item()
            iter_loss.append(loss_value)
            test_loss+=loss_value
            acc=compute_accuray(output,target)
            test_acc+=acc
        test_losses.append(test_loss/len(test_loader))
        print("Epoch {}: train loss is {}, train accuracy is {}; test loss is {}, test accuracy is {}".
              format(epoch,round(train_loss/len(train_loader),2),
                     round(train_acc/len(train_loader),2),
                     round(test_loss/len(test_loader),2),
                     round(test_acc/len(test_loader),2)))        
        if test_loss/len(test_loader)<last_loss:      
            save_model_path=os.path.join(out_dir,"models","best_model.tar".format(epoch))
            torch.save({
                    "model":m.state_dict(),
                    "optimizer":optimizer.state_dict()
                },save_model_path)
            last_loss=test_loss/len(test_loader)
        
    df=pd.DataFrame()
    df["iteration"]=np.arange(0,len(iter_loss))
    df["loss"]=iter_loss
    df.to_csv(iter_loss_path,index=False)
    
    df=pd.DataFrame()
    df["epoch"]=np.arange(0,nb_epochs)
    df["train_loss"]=train_losses
    df["test_loss"]=test_losses
    df.to_csv(epoch_loss_path,index=False)    

In [10]:
train(knn,"lenet-knn-cp0.3")

Epoch 0: train loss is 1.79, train accuracy is 0.42; test loss is 0.6, test accuracy is 0.89
Epoch 1: train loss is 0.67, train accuracy is 0.8; test loss is 0.2, test accuracy is 0.95
Epoch 2: train loss is 0.44, train accuracy is 0.87; test loss is 0.13, test accuracy is 0.96
Epoch 3: train loss is 0.35, train accuracy is 0.9; test loss is 0.1, test accuracy is 0.97
Epoch 4: train loss is 0.3, train accuracy is 0.91; test loss is 0.08, test accuracy is 0.97
Epoch 5: train loss is 0.27, train accuracy is 0.92; test loss is 0.08, test accuracy is 0.98
Epoch 6: train loss is 0.24, train accuracy is 0.93; test loss is 0.07, test accuracy is 0.98
Epoch 7: train loss is 0.22, train accuracy is 0.94; test loss is 0.07, test accuracy is 0.98
Epoch 8: train loss is 0.21, train accuracy is 0.94; test loss is 0.06, test accuracy is 0.98
Epoch 9: train loss is 0.19, train accuracy is 0.94; test loss is 0.06, test accuracy is 0.98
Epoch 10: train loss is 0.19, train accuracy is 0.94; test loss is